In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [52]:
data = pd.read_csv(r'./Data/Animal.csv')
data

,Animal,Age,Worth,Friendly
0,Cat,4.0,72000.0,No
1,Dog,17.0,48000.0,Yes
2,Moose,6.0,54000.0,No
3,Dog,8.0,61000.0,No
4,Moose,4.0,NaN,Yes
5,Cat,15.0,58000.0,Yes
6,Dog,NaN,52000.0,No
7,Cat,12.0,79000.0,Yes
8,Moose,5.0,83000.0,No
9,Cat,7.0,67000.0,Yes


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
Animal      10 non-null object
Age         9 non-null float64
Worth       9 non-null float64
Friendly    10 non-null object
dtypes: float64(2), object(2)
memory usage: 400.0+ bytes


In [54]:
data.describe()

,Age,Worth
count,9.000000,9.000000
mean,8.666667,63777.777778
std,4.847680,12265.579662
min,4.000000,48000.000000
25%,5.000000,54000.000000
50%,7.000000,61000.000000
75%,12.000000,72000.000000
max,17.000000,83000.000000


## 数据预处理

### 划分数据集

In [55]:
# 划分数据集
# 第一个冒号表示行索引，冒号两边可以输入数字表示取行的范围
# 第二个冒号表示列索引，和行同理
x = data.iloc[:, :-1].values
y = data.iloc[:, 3].values
x, y

(array([['Cat', 4.0, 72000.0],
        ['Dog', 17.0, 48000.0],
        ['Moose', 6.0, 54000.0],
        ['Dog', 8.0, 61000.0],
        ['Moose', 4.0, nan],
        ['Cat', 15.0, 58000.0],
        ['Dog', nan, 52000.0],
        ['Cat', 12.0, 79000.0],
        ['Moose', 5.0, 83000.0],
        ['Cat', 7.0, 67000.0]], dtype=object),
 array(['No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes'],
       dtype=object))

### 缺失值处理

* 最常用的方法是，用其所在列的均值来填充缺失。
* 可以利用 scikit-learn 预处理模型中的 inputer 类实现。

In [56]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values=np.nan, strategy='mean', axis=0)
# missing_values 表示缺失值取值NaN
# strategy 表示填补缺失值策略，这里使用mean，即均值

In [57]:
# 在自变量中，只用第2，3列出现缺失值
# 因此仅拟合有缺失值的列
imputer = imputer.fit(x[:, 1:3])

# 填补缺失值
x[:,1:3]=imputer.transform(x[:, 1:3])
x

array([['Cat', 4.0, 72000.0],
       ['Dog', 17.0, 48000.0],
       ['Moose', 6.0, 54000.0],
       ['Dog', 8.0, 61000.0],
       ['Moose', 4.0, 63777.77777777778],
       ['Cat', 15.0, 58000.0],
       ['Dog', 8.666666666666666, 52000.0],
       ['Cat', 12.0, 79000.0],
       ['Moose', 5.0, 83000.0],
       ['Cat', 7.0, 67000.0]], dtype=object)

使用缺失值所在列的**中位数**或**众数**来填充缺失值会更加合理。填充策略之类的决策看似细微，但其实意义重大。因为流行通用的方法并不一定就是正确的选择，对于模型而言，均值**也不一定**是最优的缺失填充选择。

### 非数值型数据编码

In [58]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
x[:, 0] = labelencoder_X.fit_transform(x[:, 0])
x

array([[0, 4.0, 72000.0],
       [1, 17.0, 48000.0],
       [2, 6.0, 54000.0],
       [1, 8.0, 61000.0],
       [2, 4.0, 63777.77777777778],
       [0, 15.0, 58000.0],
       [1, 8.666666666666666, 52000.0],
       [0, 12.0, 79000.0],
       [2, 5.0, 83000.0],
       [0, 7.0, 67000.0]], dtype=object)

该编码方式存在问题，即将文本转换成对应的编码，会让模型识别编码是一个数而不是一个类，因此最好使用one-hot编码方式

In [59]:
from sklearn.preprocessing import OneHotEncoder
onehot_x=OneHotEncoder(categorical_features = [0])
x=onehot_x.fit_transform(x).toarray()
x

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.00000000e+00,
        7.20000000e+04],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.70000000e+01,
        4.80000000e+04],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 6.00000000e+00,
        5.40000000e+04],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 8.00000000e+00,
        6.10000000e+04],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 4.00000000e+00,
        6.37777778e+04],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.50000000e+01,
        5.80000000e+04],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 8.66666667e+00,
        5.20000000e+04],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.20000000e+01,
        7.90000000e+04],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 5.00000000e+00,
        8.30000000e+04],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.00000000e+00,
        6.70000000e+04]])

In [61]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

## 测试集和训练集的划分